# Лабораторная работа №8

**Студент:** Феоктистов Владислав Сергеевич

**Группа:** НПМбд-01-19б

## Цель работы 

Освоить на практике применение режима однократного гаммирования на примере кодирования различных исходных текстов одним ключом.

## Теория

Предложенная Г. С. Вернамом так называемая «схема однократного использования (гаммирования)» (рис. 7.1) является простой, но надёжной схемой шифрования данных.

Гаммирование представляет собой наложение (снятие) на открытые (зашифрованные) данные последовательности элементов других данных, полученной с помощью некоторого криптографического алгоритма, для получения зашифрованных (открытых) данных. Иными словами, наложение гаммы — это сложение её элементов с элементами открытого (закрытого) текста по некоторому фиксированному модулю, значение которого представляет собой известную часть алгоритма шифрования.

В соответствии с теорией криптоанализа, если в методе шифрования используется однократная вероятностная гамма (однократное гаммирование) той же длины, что и подлежащий сокрытию текст, то текст нельзя раскрыть. Даже при раскрытии части последовательности гаммы нельзя получить информацию о всём скрываемом тексте.

Наложение гаммы по сути представляет собой выполнение операции сложения по модулю 2 (XOR) (обозначаемая знаком ⊕) между элементами гаммы и элементами подлежащего сокрытию текста. Напомним, как работает операция XOR над битами: 0 ⊕ 0 = 0, 0 ⊕ 1 = 1, 1 ⊕ 0 = 1, 1 ⊕ 1 = 0.

Такой метод шифрования является симметричным, так как двойное прибавление одной и той же величины по модулю 2 восстанавливает исход значение, а шифрование и расшифрование выполняется одной и той же программой.
Если известны ключ и открытый текст, то задача нахождения шифротекста заключается в применении к каждому символу открытого текста следующего правила:

$$C_i = P_i \oplus K_i$$

где $C_i$ — i-й символ получившегося зашифрованного послания, $P_i$ — i-й символ открытого текста, $K_i$ — i-й символ ключа, $i = \overline{1,m}$. Размерности открытого текста и ключа должны совпадать, и полученный шифротекст будет такой же длины.

Если известны шифротекст и открытый текст, то задача нахождения ключа сводится к повторному сложению по модулю 2 $C_i$ с $P_i$:

$$C_i \oplus P_i = P_i \oplus K_i \oplus P_i = K_i$$

$$K_i = C_i \oplus P_i$$

Открытый текст имеет символьный вид, а ключ — шестнадцатеричное представление. Ключ также можно представить в символьном виде, воспользовавшись таблицей ASCII-кодов.

К. Шеннон доказал абсолютную стойкость шифра в случае, когда однократно используемый ключ, длиной, равной длине исходного сообщения, является фрагментом истинно случайной двоичной последовательности с равномерным законом распределения. Криптоалгоритм не даёт никакой информации об открытом тексте: при известном зашифрованном сообщении $C$ все различные ключевые последовательности $K$ возможны и равновероятны, а значит, возможны и любые сообщения $P$.

Необходимые и достаточные условия абсолютной стойкости шифра:
- полная случайность ключа;
- равенство длин ключа и открытого текста;
- однократное использование ключа.

## Порядок выполнения работы

Два текста кодируются одним ключом (однократное гаммирование).
Требуется не зная ключа и не стремясь его определить, прочитать оба текста. Необходимо разработать приложение, позволяющее шифровать и дешифровать тексты P1 и P2 в режиме однократного гаммирования. Приложение должно определить вид шифротекстов C1 и C2 обоих текстов P1 и P2 при известном ключе ; Необходимо определить и выразить аналитически способ, при котором злоумышленник может прочитать оба текста, не зная ключа и не стремясь его определить

## Выполнение лабораторной работы

### Написание вспомогательных функций

Для начала загрузим необходимые для работы библиотеки:

In [1]:
from random import randint

Напишем функцию, представляющее строку в виде строки последовательности символов в шестнадцатиричном виде, разделенных двоеточием.

In [2]:
def str_to_hex(string):
    return ":".join("{:02x}".format(ord(c)) for c in string)

def hex_to_srt(string):
    return "".join(chr(int(h, 16)) for h in string.split(':'))

Напишем функцию дополнения текста пробелами в конце до нужной длины текста.

In [3]:
def add_endspaces(text, required_lenght):
    if len(text) > required_lenght:
        raise Exception('Требуемая длина меньше длины исходного текста!')
    return text + (' ' * (required_lenght - len(text)))

Напишем функцию, позволяющую шифровать и дешифровать данные в режиме однократного гаммирования.

In [4]:
def gamming(text, key):
    """
    Функция однократного гаммирования открытого текста по ключу гаммирования.
    Зашифрованный текст получается путем поэлементного сложения по модулю 2 открытого текст и ключа.
    :param text: открытый текст, подлежащий сокрытию (шифрованию)
    :param key: ключ для гаммирования
    
    :return: зашифрованный текст
    """
    if len(text) != len(key):
        raise Exception('Длина шифруемого текста должна совпадать с длиной ключа (открытого текста)!')
    return ''.join(chr(ord(t) ^ ord(k)) for t,k in zip(text, key))

Проверим работу функции. В качестве открытого текста для шифрования используем сообщение "Hello world!", а в качестве ключа сообщение "Just message" (ключ может быть и случайным набором символов той же длины, что и открытый текст).

In [5]:
text = 'Hello world!'
key = 'Just message'

encrypted_text = gamming(text, key)
decrypted_text = gamming(encrypted_text, key)

print(encrypted_text)
print(decrypted_text)

D
Hello world!


Как видно, повторное гаммирование тем же ключом дало исходный текст.

Напишем функцию, которая генерирует случайную последовательность символов из латиницы заданной длины. Вместо латицинцы можно использовать и другие символы, это необходимо, чтобы открытый текст выглядил более-менее похожим на реальный текст.

In [6]:
def generate_random_str(lenght):
    """
    Функция возарщает строку со случайной последовательностью 
    латинских символов с пробелами.
    :param lenght: длина желаемой строки
    
    :return: строка со случайной последовательностью 
    латинских символов с пробелами
    """
    letters = ' abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return ''.join(letters[randint(0, len(letters) - 1)] for i in range(lenght))

### Получение второго исходного текста

Пусть у центра есть две телеграммы $P_1$ и $P_2$, а также ключ $K$. Тогда с помощью этого ключа можно зашифровать эти телеграммы методом однокоратного гаммирования:

$$C_1 = P_1 \oplus K$$

$$C_2 = P_2 \oplus K$$

Открытый текст можно найти в соответствии с формулой выше, зная шифротекст двух телеграмм, зашифрованных одним ключом. Для это оба равенства складываются по модулю 2. Тогда с учётом свойства операции XOR

$$1 \oplus 1 = 0, 1 \oplus 0 = 1$$

получаем:

$$C_1 \oplus C_2 = P_1 \oplus K \oplus P_2 \oplus K = P_1 \oplus P_2$$

Предположим, что одна из телеграмм является шаблоном — т.е. имеет текст фиксированный формат, в который вписываются значения полей.

Допустим, что злоумышленнику этот формат известен. Тогда он получает достаточно много пар $C_1 \oplus C_2$ (известен вид обеих шифровок). Тогда зная $P_1$ и учитывая, что $C_1 \oplus C_2 = P_1 \oplus P_2$, имеем:

$$C_1 \oplus C_2 \oplus P_1 = P_1 \oplus P_2 \oplus P_1 = P_2$$

Таким образом, злоумышленник получает возможность определить те символы сообщения P2, которые находятся на позициях известного шаблона сообщения P1. В соответствии с логикой сообщения P2, злоумышленник имеет реальный шанс узнать ещё некоторое количество символов сообщения P2. Затем вновь используется выведенная формула с подстановкой вместо P1 полученных на предыдущем шаге новых символов сообщения P2. И так далее.

Действуя подобным образом, злоумышленник даже если не прочитает оба сообщения, то значительно уменьшит пространство их поиска.

Сымитируем похожую ситуацию, используя написанные выше функции.

In [9]:
P1 = 'Новости: В московском метро появился новый тематический поезд «Сибирь здесь»'
P2 = 'Южный речной вокзал в Москве планируют открыть к лету 2023 года'

P1_hacked = 'Новости: '
P2_hacked = ''
k = 0
prev_k = k

lenght = max(len(P1), len(P2))
P1 = add_endspaces(P1, lenght)
P2 = add_endspaces(P2, lenght)
K = generate_random_str(lenght)

C1 = gamming(P1, K)
C2 = gamming(P2, K)
C = gamming(C1, C2)

while (len(P1_hacked) != lenght and len(P2_hacked) != lenght):
    
    if k % 2 == 0:
        P2_hacked_tmp = gamming(C[:len(P1_hacked)], P1_hacked)
        print('\nУдалось расшифровать часть второго сообщения:', P2_hacked_tmp)
        if k != prev_k:
            if input('Back? [y/n]: ') == 'y':
                k -= 1
                P1_hacked = P1_hacked_prev
                continue
        P2_hacked = P2_hacked_tmp
        P2_hacked_prev = P2_hacked
        P2_hacked += input('Введите предполагаемое продолжение для второго сообщения:')
        print('Теперь второе сообщение имеет вид:', P2_hacked)
    else:
        P1_hacked_tmp = gamming(C[:len(P2_hacked)], P2_hacked)
        print('\nУдалось расшифровать часть первого сообщения:', P1_hacked_tmp)
        if k != prev_k:
            if input('Back? [y/n]: ') == 'y':
                k -= 1
                P2_hacked = P2_hacked_prev
                continue
        P1_hacked = P1_hacked_tmp
        P1_hacked_prev = P1_hacked
        P1_hacked += input('Введите предполагаемое продолжение для первого сообщения:')
        print('Теперь первое сообщение имеет вид:', P1_hacked)
    
    if input('Continue? [y/n]: ') == 'n':
        break
    prev_k = k
    k += 1
 
if len(P1_hacked) == lenght and len(P2_hacked) == lenght:
    print('Два сообщения были полностью расшифрованы!')


Удалось расшифровать часть второго сообщения: Южный реч
Введите предполагаемое продолжение для второго сообщения:ной вокзал 
Теперь второе сообщение имеет вид: Южный речной вокзал 
Continue? [y/n]:y

Удалось расшифровать часть первого сообщения: Новости: В московско
Back? [y/n]:n
Введите предполагаемое продолжение для первого сообщения:м зоопарке 
Теперь первое сообщение имеет вид: Новости: В московском зоопарке 
Continue? [y/n]:y

Удалось расшифровать часть второго сообщения: Южный речной вокзал в ЗенхмU%дT
Back? [y/n]:y

Удалось расшифровать часть первого сообщения: Новости: В московско
Введите предполагаемое продолжение для первого сообщения:м метро
Теперь первое сообщение имеет вид: Новости: В московском метро
Continue? [y/n]:y

Удалось расшифровать часть второго сообщения: Южный речной вокзал в Москв
Back? [y/n]:n
Введите предполагаемое продолжение для второго сообщения:е 
Теперь второе сообщение имеет вид: Южный речной вокзал в Москве 
Continue? [y/n]:y

Удалось расшифровать час

Exception: Длина шифруемого текста должна совпадать с длиной ключа (открытого текста)!

Таким образом, зная лишь только начало первого сообщения, получилось расшифровать все два сообщения.

## Выводы

- Освоина на практике применение режима однократного гаммирования;
- Написана программа для гаммирования текст и по совместительству для нахождения ключа, дающего желаемое сообщение. 

## Контрольные вопросы

**1. Как, зная один из текстов (P1 или P2), определить другой, не зная при этом ключа?**
   
Допустим, что злоумышленнику известен полностью текст $P_1$. Тогда он получает достаточно много пар $C_1 \oplus C_2$ (известен вид обеих шифровок). Тогда зная учитывая, что $C_1 \oplus C_2 = P_1 \oplus P_2$, имеем:

$$C_1 \oplus C_2 \oplus P_1 = P_1 \oplus P_2 \oplus P_1 = P_2$$

Таким образом, злоумышленник может полностью получить второй текст $P_2$. При этом для этого даже не нужно находить ключ. Таким же способом можно найти вест текст $P_1$, если известен полностью текст $P_2$. Если же известна часть только одного текста, то можно получать символы постепенно, расшифровывая один за счет другого и добавляя возможную последовательность символов. 
   
**2. Что будет при повторном использовании ключа при шифровании текста?**

При повторном использовании ключа в соответствии с формулами:

$$C_i = P_i \oplus K_i$$

$$C_i \oplus K_i = P_i \oplus K_i \oplus K_i = P_i$$

получаем исходный текст.

**3. Как реализуется режим шифрования однократного гаммирования одним ключом двух открытых текстов?**

Режим шифрования однократного гаммирования одним ключом двух видов открытого текста реализуется в соответствии с формулами и со схемой ниже.

Пусть у центра есть две телеграммы $P_1$ и $P_2$, а также ключ $K$. Тогда с помощью этого ключа можно зашифровать эти телеграммы методом однокоратного гаммирования:

$$C_1 = P_1 \oplus K$$

$$C_2 = P_2 \oplus K$$

СХЕМА

**4. Перечислите недостатки шифрования одним ключом двух открытых текстов.**

Недостатки шифрования одним ключом двух открытых текстов:

- зная полностью одно сообщение, можно получить и второе;
- зная лишь часть одного сообщения, возможно постепенно расшифровать оба сообщения;
- необходимо, чтобы оба сообщения были одинаковых размеров.

**5. Перечислите преимущества шифрования одним ключом двух открытых текстов.**

Преимущества шифрования одним ключом двух открытых текстов:

- из преимущества только немного сэкономленная память, однако это полностью невелируется остутсвием достаточной безопасности.